In [1]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.0.1.


In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
data.target.sum() #165 человек у которых таргет = 1
#Всего 303 вхождения, соответственно 165/303 - процент с заболвением

165

In [79]:
165/303 #человеки с заболеванием сердца

0.5445544554455446

In [2]:
a = data.target.count()
b = data.target.sum()
c = b/a
c

NameError: name 'data' is not defined

In [7]:
from sklearn.linear_model import SGDClassifier

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=0.25, random_state=13)
X_train.shape, X_test.shape

((227, 13), (76, 13))

In [9]:
clf = SGDClassifier(max_iter=1000, learning_rate='constant', eta0=0.1, random_state=13, alpha=0, loss='log')

In [10]:
clf.fit(X_train, y_train)

SGDClassifier(alpha=0, eta0=0.1, learning_rate='constant', loss='log',
              random_state=13)

In [11]:
clf.coef_ #веса при признаках

array([[-93.18252315, -18.25927929,  33.13144142, -65.99603819,
          1.75342142,   2.24072071,   6.44072071, 112.79675487,
        -16.1       , -33.35463964,  11.73144141, -38.3       ,
        -25.37783787]])

In [12]:
clf.intercept_ #свободный коэф

array([-1.15927929])

In [13]:
coef = clf.coef_.copy()

L2-коэф - корень из суммы квадратов коэфициентов весов.
Соотв находим сумму квардратов, а потом находим корень

In [14]:
forl = 0
for i in coef:
    for a in i:
        forl += a**2
print(forl)


30862.53283430207


In [15]:
from math import sqrt
l2 = sqrt(forl)
l2

175.67735435821564

In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, clf.predict(X_test))

0.618421052631579

In [17]:
clf_ch = SGDClassifier(loss='log', max_iter=1000, penalty='l1', learning_rate='optimal', alpha=0.1, random_state=13)
clf_ch.fit(X_train, y_train)

SGDClassifier(alpha=0.1, loss='log', penalty='l1', random_state=13)

Сравнение получившихся коэфициентов 

In [18]:
clf.coef_, clf.intercept_ 

(array([[-93.18252315, -18.25927929,  33.13144142, -65.99603819,
           1.75342142,   2.24072071,   6.44072071, 112.79675487,
         -16.1       , -33.35463964,  11.73144141, -38.3       ,
         -25.37783787]]),
 array([-1.15927929]))

In [19]:
clf_ch.coef_, clf_ch.intercept_

(array([[-0.87424016,  0.        ,  9.26085004, -0.45773776, -0.01501012,
          0.        ,  0.        ,  0.74467772,  0.        , -0.25903021,
          0.        , -4.1192229 , -2.1071261 ]]),
 array([-0.56631733]))

In [20]:
accuracy_score(y_test, clf_ch.predict(X_test))

0.7631578947368421

In [21]:
coef_new = clf_ch.coef_.copy()

forl_new = 0
for i in coef_new:
    for a in i:
        forl_new += a**2
print(forl_new, 'Сумма квадратов коэфициентов')

l2_new = sqrt(forl_new)
print(l2_new, 'Итоговая L2-норма')

108.76700763224905 Сумма квадратов коэфициентов
10.429142228977849 Итоговая L2-норма


In [22]:
arr = []

for sign in clf.coef_:
    for sig in sign:
        arr.append(sig)


In [25]:
proba = clf.predict_proba(X_test)

In [64]:
plusprob = proba[:, 1] #достаем положительный столбец

In [65]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, plusprob)

0.5697674418604651

В документации auc+pred_prob показаны так же

In [67]:
roc_auc_score(y_test, clf_ch.predict_proba(X_test)[:, 1])

0.8558844256518675

thalach (максимальный bps сердца) имеет самый большой вес в коэфициентах для первого классификатора
Для второго sex (пол)